# 🎯 Geodata and Mapplots

In [ ]:
import pandas as pd
import numpy as np 
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
import contextily as cx
from collections import Counter
import matplotlib.font_manager
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
df = pd.read_csv("C:/Data/DSF Project/NigeriaCleanedFinal.csv")

Adding a concatenated location Column

In [ ]:
df["location_append"] = df["Area"] + ", " + df["State"]
df["Price Standard"] = (df["Price"]-np.mean(df["Price"]))/np.std(df["Price"])

Create a dataframe by areas with desired metrics

In [ ]:
df_plt = pd.DataFrame()
df_plt["Area"] = df["location_append"].unique()
df_plt["Count"] = Counter(df["location_append"]).values()


medprice = []
medpricestd = []
meanbed = []
meanbath = []
pernew = []

for i in df_plt["Area"]:
    medprice.append(np.median(df.loc[df["location_append"] == i,["Price"]]))
    medpricestd.append(np.median(df.loc[df["location_append"] == i,["Price Standard"]]))
    meanbed.append(np.array(df.loc[df["location_append"] == i,["Bedrooms"]]).mean())
    meanbath.append(np.array(df.loc[df["location_append"] == i,["Bathrooms"]]).mean())
    pernew.append((np.array(df.loc[df["location_append"] == i,["Newly Built"]]).sum())/(df.loc[df["location_append"] == i,["Newly Built"]].shape[0]))
    
df_plt["Median Rental Price"] = medprice
df_plt["Median Rental Price Standard"] = medpricestd  
df_plt["Mean Bedrooms"] = meanbed 
df_plt["Mean Bathrooms"] = meanbath
df_plt["Newly Built Percentage"] = pernew

Created a Polygon Shapefile in QGIS based on the Area Classifications used in the Data Set. Import File and combine with Dataframe to add Geodata for each Area. Leaving out Ojo due to inconclusiveness.

In [ ]:
#Import Polygon Shapefile and combine with current Dataframe
Lagos = gpd.read_file("C:/Data/DSF Project/Lagos/Lagos Areas.shp")
Lagos["Area"] = Lagos["Name"] + ", Lagos"

#Merge Dataframes and delete irrelevant information
gdf = pd.merge(df_plt, Lagos, how="inner", on="Area")

del gdf["id"]
del gdf["Name"]

#Convert into Geopandas Dataframe and extract centroid coordinates
gdf = gpd.GeoDataFrame(gdf) 

gdf.to_crs('+proj=cea')

gdf["x"] = gdf.centroid.x
gdf["y"] = gdf.centroid.y

Implementing acquired Geodata over original Dataset adding coordinates for Machine Learning Purposes

In [ ]:
long = {}
lat = {}

for i in gdf["Area"]:
    long[i] = gdf.loc[gdf["Area"] == i, ["x"]].x.item()
    lat[i] = gdf.loc[gdf["Area"] == i, ["y"]].y.item()

loncol = []
latcol = []

for i in df["location_append"]:
        loncol.append(long[i])
        latcol.append(lat[i])

df["Longitude"] = loncol
df["Latitude"] = latcol

In [ ]:
#df.to_csv("C:/Data/DSF Project/NigeriaCleanedFinalCoordinates.csv") 

Plots:

In [ ]:
#Plotting median standardized rental prices by are and mean bedrooms per area
def MapPlotNarrow(df, a, b): 
    
    gdf_plt = gdf.loc[gdf["Count"] >= 4]

    fig, axs = plt.subplots(1,2,figsize = (9,6))

    fig.tight_layout(pad=5)

    plt.rcParams["font.family"] = "Arial Nova"
    
    cols = [a,b]
    n = -1


    for i in cols:

        n += 1
    
        axs[n].set_xlim(3.2,3.75)
        axs[n].set_ylim(6.35,6.7)
        axs[n].set_title(f"{i} per Area")

        #Aligning Colorbar to Axis
        divider = make_axes_locatable(axs[n])
        cax = divider.append_axes("right", size="5%", pad=0.1)


        gdf_plt.plot(ax=axs[n], column= i,cmap="Blues", alpha = 0.8, edgecolor='k', linewidth=0.1, legend=True, 
         legend_kwds={"cax" : cax })
    
        #Adding Basemap
        cx.add_basemap(ax=axs[n], source=cx.providers.Stamen.TonerLite, crs=gdf.crs.to_string())
        
def MapPlotWide(df, a, b): 
    
    gdf_plt = gdf.loc[gdf["Count"] >= 1]

    fig, axs = plt.subplots(1,2,figsize = (15,12))

    fig.tight_layout(pad=5)
    
    plt.rcParams["font.family"] = "Arial Nova"

    cols = [a,b]
    n = -1


    for i in cols:

        n += 1
    
        axs[n].set_xlim(2.8,4.2)
        axs[n].set_ylim(6.35,6.7)
        axs[n].set_title(f"{i} per Area")

        #Aligning Colorbar to Axis
        divider = make_axes_locatable(axs[n])
        cax = divider.append_axes("right", size="5%", pad=0.1)


        gdf_plt.plot(ax=axs[n], column= i,cmap="Blues", alpha = 0.8, edgecolor='k', linewidth=0.1, legend=True, 
         legend_kwds={"cax" : cax })
    
        #Adding Basemap
        cx.add_basemap(ax=axs[n], source=cx.providers.Stamen.TonerLite, crs=gdf.crs.to_string())


In [ ]:
MapPlotWide(gdf, "Median Rental Price", "Count")

In [ ]:
MapPlotNarrow(gdf, "Mean Bedrooms", "Newly Built Percentage")

Plots for the Paper

In [ ]:
#Plots for Paper v.1
gdf_plt = gdf.loc[gdf["Count"] >= 1]

fig, axs = plt.subplots(1,2,figsize = (20,20))

fig.tight_layout(pad=5)

plt.rcParams["font.family"] = "Arial Nova"
    
axs[0].set_xlim(2.8,4.2)
axs[0].set_ylim(6.35,6.7)
axs[0].set_title("Median Rental Price per Area",  size='20')

axs[1].set_xlim(2.8,4.2)
axs[1].set_ylim(6.35,6.7)
axs[1].set_title("Count per Area",  size='20')

#Aligning Colorbar to Axis
divider0 = make_axes_locatable(axs[0])
cax0 = divider0.append_axes("right", size="5%", pad=0.1)

divider1 = make_axes_locatable(axs[1])
cax1 = divider1.append_axes("right", size="5%", pad=0.1)

#Plots
gdf_plt.plot(ax=axs[0], column= "Median Rental Price",cmap="Blues", alpha = 0.8, edgecolor='k', linewidth=0.1, legend=True, 
         legend_kwds={"cax" : cax0 })
gdf_plt.plot(ax=axs[1], column= "Count",cmap="Blues",vmin = 0, vmax = 500, alpha = 0.8, edgecolor='k', linewidth=0.1, legend=True, 
         legend_kwds={"cax" : cax1 })    


#Adding Basemap
cx.add_basemap(ax=axs[0], source=cx.providers.Stamen.TonerLite, crs=gdf.crs.to_string())
cx.add_basemap(ax=axs[1], source=cx.providers.Stamen.TonerLite, crs=gdf.crs.to_string())

#plt.savefig('C:/Data/DSF Project/map_wide.png')

In [ ]:
#Plots for Paper v.2
gdf_plt = gdf.loc[gdf["Count"] >= 1]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 20))

fig.tight_layout(pad=5)

plt.subplots_adjust(wspace=0, hspace=-0.5)

plt.rcParams['font.family'] = 'Arial Nova'
    
ax1.set_xlim(3.1,4.2)
ax1.set_ylim(6.37,6.7)
ax1.set_title("Median Rental Price per Area", font='Arial Nova', size='20')

ax2.set_xlim(3.1,4.2)
ax2.set_ylim(6.37,6.7)
ax2.set_title("Count per Area", size='20')

#Aligning Colorbar to Axis
divider0 = make_axes_locatable(ax1)
cax0 = divider0.append_axes("right", size="5%", pad=0.1)

divider1 = make_axes_locatable(ax2)
cax1 = divider1.append_axes("right", size="5%", pad=0.1)

#Plots
gdf_plt.plot(ax=ax1, column= "Median Rental Price",cmap="Blues", alpha = 0.8, edgecolor='k', legend=True, 
         legend_kwds={"cax" : cax0 }, linewidth=0.1)
gdf_plt.plot(ax=ax2, column= "Count",cmap="Blues",vmin = 0, vmax = 500, alpha = 0.8, edgecolor='k',linewidth=0.1, legend=True, 
         legend_kwds={"cax" : cax1 })    


#Adding Basemap
cx.add_basemap(ax=ax1, source=cx.providers.Stamen.TonerLite, crs=gdf.crs.to_string())
cx.add_basemap(ax=ax2, source=cx.providers.Stamen.TonerLite, crs=gdf.crs.to_string())

#plt.savefig('C:/Data/DSF Project/map_paper.png')